<a href="https://colab.research.google.com/github/vichruth/LANE-DETECTION/blob/main/transferlanedetection(using%20transfer%20learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-segmentation", model="bricklerex/lane-detect-jds")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.9M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [2]:
# Load model directly
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation

processor = AutoImageProcessor.from_pretrained("bricklerex/lane-detect-jds")
model = SegformerForSemanticSegmentation.from_pretrained("bricklerex/lane-detect-jds")

In [11]:
from moviepy.editor import VideoFileClip
from PIL import Image
import numpy as np
import torch

def process_frame(frame):
    img = Image.fromarray(frame).resize((512, 512))
    inputs = processor(images=img, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    mask = outputs.logits.argmax(dim=1)[0].cpu().numpy()

    colored_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    colored_mask[mask == 1] = [0, 255, 0]

    # Resize the colored_mask to match the size of the resized image
    colored_mask_resized = Image.fromarray(colored_mask).resize((512, 512))
    colored_mask_resized = np.array(colored_mask_resized)

    overlay = (0.6 * np.array(img) + 0.4 * colored_mask_resized).astype(np.uint8)
    return overlay

clip = VideoFileClip("lanes_clip.mp4")
processed_clip = clip.fl_image(process_frame)
processed_clip.write_videofile("lane_output.mp4", audio=False)

Moviepy - Building video lane_output.mp4.
Moviepy - Writing video lane_output.mp4



Moviepy - Done !
Moviepy - video ready lane_output.mp4


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
!ls /content/drive/MyDrive/Colab\ Notebooks

'audio classifier.ipynb'			    task2.ipynb
'C1_W1_Lab03_Model_Representation_Soln (1).ipynb'  'task(remade).ipynb'
 C1_W1_Lab03_Model_Representation_Soln.ipynb	    transferlanedetection.ipynb
'cnn image classification.ipynb'		    Untitled
'Copy of vadivelu.ipynb'			    Untitled0.ipynb
'lane detection.ipynb'				    Untitled1.ipynb
'task1(bonus).ipynb'				    yolo.ipynb
 task1.ipynb


In [31]:
import nbformat

# Full correct path (note the space in 'Colab Notebooks')
notebook_path = "/content/drive/MyDrive/Colab Notebooks/transferlanedetection.ipynb"

# Load and clean notebook metadata
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

with open(notebook_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print("✅ Notebook metadata cleaned successfully!")



✅ Notebook metadata cleaned successfully!
